# ids projekti, kivaa :---)

tästä alkaa suuri seikkailu. Olkaamme innokaita ja tehokkaita. 

In [1]:
import json
import pandas as pd
import numpy as np
import urllib.request as ur
import requests
from datetime import timedelta, date
from sklearn.model_selection import train_test_split as tts
from tpot import TPOTClassifier

In [317]:
def authorize():
    path = 'APIauth.txt'
    temp = open(path,'r').read().splitlines()
    return (temp[0], temp[1])

In [2]:

def date_range(n):
    date_list = []

    for single_date in (date.today() - timedelta(n+1) for n in range(n)):
        date_list.append(single_date.strftime('%Y%m%d'))

    return(date_list)

In [300]:

def fetch_game_scoreboards():
    games_lists = []

    un, pw = authorize()

    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/scoreboard.json?fordate={date}'
    
    DATE_RANGE = 100
    n=0
    for i in date_range(DATE_RANGE):
        if n%50 == 0:
            print(n)
            
        n += 1
        
        try:
            json = requests.get(url.format(date=i), auth=(un, pw)).json()
            json['scoreboard']['gameScore']
        except:
            continue
        
        games_lists.append(json['scoreboard']['gameScore'])
        
       

    return(games_lists)

In [318]:
def fetch_team_data():
    un, pw = authorize()
    
    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/overall_team_standings.json'
    
    json = requests.get(url, auth=(un,pw)).json()
    
    data = json['overallteamstandings']['teamstandingsentry']
    
    return(data)

In [356]:
#LATAA PELIT
#TÄSSÄ KESTÄÄ KAUAN, KANNATTAA TEHDÄ VAIN KERRAN

raw_game_data = fetch_game_scoreboards()

game_data = np.hstack(raw_game_data)

print(game_data.shape)

0
50
(148,)


In [320]:
team_data = np.hstack(fetch_team_data())

print(team_data)

[ {'team': {'ID': '1', 'City': 'Tampa Bay', 'Name': 'Lightning', 'Abbreviation': 'TBL'}, 'rank': '1', 'stats': {'GamesPlayed': {'@abbreviation': 'GP', '#text': '10'}, 'stats': {'Wins': {'@abbreviation': 'W', '#text': '8'}, 'Losses': {'@abbreviation': 'L', '#text': '1'}, 'OvertimeWins': {'@abbreviation': 'OTW', '#text': '0'}, 'OvertimeLosses': {'@abbreviation': 'OTL', '#text': '1'}, 'GoalsFor': {'@abbreviation': 'GF', '#text': '41'}, 'GoalsAgainst': {'@abbreviation': 'GA', '#text': '24'}, 'FaceoffWins': {'@abbreviation': 'F/O W', '#text': '309'}, 'FaceoffLosses': {'@abbreviation': 'F/O L', '#text': '329'}, 'FaceoffPercent': {'@abbreviation': 'F/O %', '#text': '48.4'}, 'Powerplays': {'@abbreviation': 'PP', '#text': '41'}, 'PowerplayGoals': {'@abbreviation': 'PPG', '#text': '12'}, 'PowerplayPercent': {'@abbreviation': 'PP%', '#text': '29.3'}, 'PenaltyKills': {'@abbreviation': 'PK', '#text': '39'}, 'PenaltyKillGoalsAllowed': {'@abbreviation': 'PKGA', '#text': '7'}, 'PenaltyKillPercent': {'

In [40]:
#Jos kotijoukkoe voittaa, winner = 1

print(team_data[0])

{'team': {'ID': '1', 'City': 'Tampa Bay', 'Name': 'Lightning', 'Abbreviation': 'TBL'}, 'rank': '1', 'stats': {'GamesPlayed': {'@abbreviation': 'GP', '#text': '10'}, 'stats': {'Wins': {'@abbreviation': 'W', '#text': '8'}, 'Losses': {'@abbreviation': 'L', '#text': '1'}, 'OvertimeWins': {'@abbreviation': 'OTW', '#text': '0'}, 'OvertimeLosses': {'@abbreviation': 'OTL', '#text': '1'}, 'GoalsFor': {'@abbreviation': 'GF', '#text': '41'}, 'GoalsAgainst': {'@abbreviation': 'GA', '#text': '24'}, 'FaceoffWins': {'@abbreviation': 'F/O W', '#text': '309'}, 'FaceoffLosses': {'@abbreviation': 'F/O L', '#text': '329'}, 'FaceoffPercent': {'@abbreviation': 'F/O %', '#text': '48.4'}, 'Powerplays': {'@abbreviation': 'PP', '#text': '41'}, 'PowerplayGoals': {'@abbreviation': 'PPG', '#text': '12'}, 'PowerplayPercent': {'@abbreviation': 'PP%', '#text': '29.3'}, 'PenaltyKills': {'@abbreviation': 'PK', '#text': '39'}, 'PenaltyKillGoalsAllowed': {'@abbreviation': 'PKGA', '#text': '7'}, 'PenaltyKillPercent': {'@a

In [125]:
from pandas.io.json import json_normalize

def fix_team_data():
    df = json_normalize(team_data)

    #test.drop(['*.@abbreviation'], axis=1, inplace=True)

    df = df[df.columns.drop(list(df.filter(regex='abbreviation|Abbreviation')))]

    df.drop(['team.Name', 'team.City'], axis=1, inplace=True)

    return(df)

team_standings = fix_team_data()


In [180]:
id_to_teamname = dict()

def parse_game(g):
    data = dict()
    
    # DEFINE IDS
    away_team_id = g['game']['awayTeam']['ID']
    home_team_id = g['game']['homeTeam']['ID']
    
    
    data['away_team_id'] = away_team_id
    data['home_team_id'] = home_team_id
    
    
    #ASSOCIATE TEAMNAMES WITH TEAM IDS
    if g['game']['awayTeam']['Abbreviation'] not in id_to_teamname:
        id_to_teamname[g['game']['awayTeam']['Abbreviation']] = g['game']['awayTeam']['ID']
        
    if g['game']['homeTeam']['Abbreviation'] not in id_to_teamname:
        id_to_teamname[g['game']['homeTeam']['Abbreviation']] = g['game']['homeTeam']['ID']
    
    
    # DEFINE WINNER
    if g['homeScore'] > g['awayScore']:
        data['winner'] = 1
    else:
        data['winner'] = 0
        
    #data['away_team_stats'] = team_standings.loc[(team_standings['team.ID'] == away_team_id)].values.flatten()
    #data['home_team_stats'] = team_standings.loc[(team_standings['team.ID'] == home_team_id)].values.flatten()

    return(pd.Series(data))


In [357]:
match_data = pd.DataFrame()


for g in game_data:
    match_data = match_data.append(parse_game(g), ignore_index=True)

print(match_data.shape)

(148, 3)


In [238]:
match_data.drop('winner', axis=1).values

array([['30', '8'],
       ['24', '10'],
       ['29', '6'],
       ['1', '3'],
       ['16', '15'],
       ['28', '13'],
       ['4', '14'],
       ['23', '18'],
       ['21', '25'],
       ['27', '22'],
       ['20', '142'],
       ['28', '12'],
       ['26', '9'],
       ['21', '16'],
       ['18', '9'],
       ['24', '6'],
       ['12', '13'],
       ['10', '1'],
       ['26', '8'],
       ['15', '11'],
       ['28', '19'],
       ['4', '5'],
       ['3', '27'],
       ['20', '30'],
       ['25', '23'],
       ['17', '142'],
       ['21', '15'],
       ['26', '7'],
       ['10', '4'],
       ['5', '16'],
       ['25', '47'],
       ['14', '29'],
       ['18', '6'],
       ['1', '19'],
       ['8', '9'],
       ['21', '11'],
       ['7', '13'],
       ['24', '20'],
       ['3', '23'],
       ['17', '22'],
       ['27', '30'],
       ['16', '12'],
       ['20', '17'],
       ['14', '28'],
       ['12', '5'],
       ['4', '6'],
       ['10', '9'],
       ['1', '7'],
       ['21', '13'

In [358]:
#team_standings.loc[(team_standings['team.ID'] == '1')]

great_match_arrays = []

d = match_data.drop('winner', axis=1).values

for i in d:
    home = i[1]
    away = i[0]
    great_match_arrays.append(np.hstack([i, team_standings.loc[(team_standings['team.ID'] == home)].values.flatten(), team_standings.loc[(team_standings['team.ID'] == away)].values.flatten()]))
    
#b = match_data.values[0]
#a = team_standings.loc[(team_standings['team.ID'] == '30')].values.flatten()
#c = team_standings.loc[(team_standings['team.ID'] == '8')].values.flatten()

final_data = np.stack(great_match_arrays)

print(final_data.shape)

(148, 52)


In [335]:
#pd.DataFrame(final_data).to_csv('final_data.csv')
target = match_data['winner'].values
pd.DataFrame(target).to_csv('target.csv')

In [359]:
def count_rookies():
    
    game_ids = []
    for data in game_data:
        game_ids.append(data['game']['ID'])

    games = {}
    
    auth = authorize()
    lineup_url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/game_startinglineup.json?gameid='
    players_json = requests.get('https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/roster_players.json?fordate=20171025', auth=auth).json()

    for id in game_ids:
        game = []
        lineup_json = requests.get(lineup_url + id, auth=auth).json()
        home_rookies = 0
        away_rookies = 0
        for lineup_player in lineup_json['gamestartinglineup']['teamLineup'][0]['expected']['starter']:
            for player in players_json['rosterplayers']['playerentry']:
                if (player['player']['ID'] == lineup_player['player']['ID']) & (player['player']['IsRookie'] == 'true'):
                    home_rookies += 1

        for lineup_player in lineup_json['gamestartinglineup']['teamLineup'][1]['expected']['starter']:
            for player in players_json['rosterplayers']['playerentry']:
                if (player['player']['ID'] == lineup_player['player']['ID']) & (player['player']['IsRookie'] == 'true'):
                    away_rookies += 1

        game.append(home_rookies)
        game.append(away_rookies)
        games[id] = game
        
        return(games)
    
rookies = count_rookies()

In [360]:
target = match_data['winner'].values

X_train, X_test, y_train, y_test = tts(final_data, target, train_size=0.75, test_size=0.25, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(111, 52) (37, 52) (111,) (37,)


In [362]:

tpot = TPOTClassifier(generations=50, population_size=50, verbosity=2, warm_start=True)
tpot.fit(X_train, y_train)


Optimization Progress:   4%|▍         | 100/2550 [00:30<13:02,  3.13pipeline/s]

Generation 1 - Current best internal CV score: 0.6308300395256916


Optimization Progress:   6%|▌         | 150/2550 [00:52<12:25,  3.22pipeline/s]

Generation 2 - Current best internal CV score: 0.6391304347826086


Optimization Progress:   8%|▊         | 200/2550 [01:22<14:24,  2.72pipeline/s]  

...

Generation 48 - Current best internal CV score: 0.6928853754940711


Optimization Progress:  98%|█████████▊| 2501/2550 [24:47<00:29,  1.64pipeline/s]

Generation 49 - Current best internal CV score: 0.6928853754940711


Generation 50 - Current best internal CV score: 0.6928853754940711

Best pipeline: DecisionTreeClassifier(RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.5, min_samples_leaf=20, min_samples_split=5, n_estimators=100), criterion=gini, max_depth=6, min_samples_leaf=16, min_samples_split=15)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=50, max_eval_time_mins=5,
        max_time_mins=None, mutation_rate=0.9, n_jobs=1, offspring_size=50,
        periodic_checkpoint_folder=None, population_size=50,
        random_state=None, scoring=None, subsample=1.0, verbosity=2,
        warm_start=True)

In [363]:

print(tpot.score(X_test, y_test))
tpot.export('DecisionTreeClassifier001.py')





0.72972972973


True

In [286]:
#print(id_to_teamname)

def predict_game(away, home):
    away_id = id_to_teamname[away]
    home_id = id_to_teamname[home]
    
    away_stats = team_standings.loc[(team_standings['team.ID'] == away_id)].values.flatten()
    home_stats = team_standings.loc[(team_standings['team.ID'] == home_id)].values.flatten()
    
    vec = np.hstack([away_id, home_id, home_stats, away_stats]).reshape(1, -1)

    res = tpot.predict(vec.reshape(1, -1))
    
    if res == 1:
        return(home)
    else:
        return(away)


In [304]:
predict_game('BUF', 'CBJ')

#print(id_to_teamname)

'CBJ'

In [355]:
#REFRESH TEAM STANDINGS

team_data = np.hstack(fetch_team_data())

print(team_data.shape)

(31,)


In [366]:
def multi_predict(m):
    ret = []
    
    for i in m:
        ret.append(predict_game(i[0], i[1]))
        
    return(ret)


predictions = [['COL', 'VGK'], ['OTT', 'NJD'], ['WPJ', 'CBJ'], ['STL', 'CAR'], ['NSH', 'CHI'], ['DAL', 'CGY']]


print(multi_predict(predictions))

['VGK', 'OTT', 'CBJ', 'STL', 'NSH', 'CGY']


In [314]:
print(id_to_teamname)

{'ARI': '30', 'NYI': '8', 'EDM': '24', 'PIT': '10', 'ANA': '29', 'PHI': '6', 'TBL': '1', 'CAR': '3', 'DET': '16', 'BUF': '15', 'LAK': '28', 'OTT': '13', 'FLO': '4', 'MTL': '14', 'CGY': '23', 'NSH': '18', 'VAN': '21', 'MIN': '25', 'DAL': '27', 'COL': '22', 'CHI': '20', 'VGK': '142', 'TOR': '12', 'SJS': '26', 'NYR': '9', 'BOS': '11', 'CBJ': '19', 'WSH': '5', 'STL': '17', 'NJD': '7', 'WPJ': '47'}


###### 27.10.17

['VGK', 'OTT', 'CBJ', 'STL', 'NSH', 'CGY']

###### 26.10.2017

SJS BOS --- BOS


CAR TOR --- TOR


ARI NYR --- NYR


WPG PIT --- PIT


LAK MTL --- LAK


PHI OTT --- PHI


DET TBL --- TBL


ANA FLA --- ANA


NYI MIN --- NYI


DAL EDM --- DAL


WSH VAN --- VAN


